In [ ]:
import requests
import pandas as pd
import json

def collect_starbucks_stores():
    """
    Collect Starbucks store information from the Korean Starbucks website API
    and convert it to a pandas DataFrame
    """
    # API endpoint
    url = "https://www.starbucks.co.kr/store/getStore.do"

    # Request payload
    payload = {
        'in_biz_cds': '0',
        'in_scodes': '0',
        'ins_lat': '37.5816192',
        'ins_lng': '126.681088',
        'search_text': '',
        'p_sido_cd': '01',
        'p_gugun_cd': '',
        'in_distance': '0',
        'in_biz_cd': '',
        'isError': 'true',
        'searchType': 'C',
        'set_date': '',
        'all_store': '0',
        'iend': '1000',
        'rndCod': 'H6HEX1ODVH'
    }

    # Add all the T* and P* parameters
    for param in ['T03', 'T01', 'T27', 'T12', 'T09', 'T30', 'T05', 'T22', 'T21',
                 'T36', 'T43', 'Z9999', 'T64', 'T66']:
        payload[param] = '0'

    for param in ['P02', 'P10', 'P50', 'P20', 'P60', 'P30', 'P70', 'P40', 'P80',
                 'P90', 'P01']:
        payload[param] = '0'

    try:
        # Make the request
        response = requests.post(url, data=payload)
        response.raise_for_status()

        # Parse the JSON response
        data = response.json()

        # Extract relevant fields for the DataFrame
        relevant_fields = [
            's_name',           # Store name
            'addr',            # Address
            'doro_address',    # Street address
            'lat',             # Latitude
            'lot',             # Longitude
            'phone',           # Phone number
            'fax',             # Fax number
            'open_dt',         # Opening date
            'gugun_name',      # District name
            's_biz_code',      # Store business code
            'my_siren_order_store_yn',  # Mobile order availability
            'defaultimage'     # Store image URL
        ]

        # Create list to store processed data
        processed_data = []

        # Process each store
        for store in data['list']:
            store_data = {}
            for field in relevant_fields:
                store_data[field] = store.get(field, None)
            processed_data.append(store_data)

        # Create DataFrame
        df = pd.DataFrame(processed_data)

        # Rename columns for clarity
        df = df.rename(columns={
            's_name': 'store_name',
            'lot': 'lng',
            'doro_address': 'street_address',
            'my_siren_order_store_yn': 'mobile_order',
            'defaultimage': 'image_url'
        })

        # Clean up datetime
        df['open_dt'] = pd.to_datetime(df['open_dt'], format='%Y%m%d', errors='coerce')

        return df

    except requests.exceptions.RequestException as e:
        print(f"Error making request: {e}")
        return None
    except json.JSONDecodeError as e:
        print(f"Error parsing JSON response: {e}")
        return None
    except Exception as e:
        print(f"Unexpected error: {e}")
        return None

# Example usage
if __name__ == "__main__":
    df = collect_starbucks_stores()
    if df is not None:
        print(f"Collected information for {len(df)} stores")
        print("\nDataFrame sample:")
        print(df.head())

        # Save to CSV (optional)
        df.to_csv('starbucks_stores_korea.csv', index=False, encoding='utf-8-sig')

Collected information for 632 stores

DataFrame sample:
  store_name                        addr              street_address  \
0    역삼아레나빌딩  서울특별시 강남구 역삼동 721-13 아레나빌딩     서울특별시 강남구 언주로 425 (역삼동)   
1     논현역사거리    서울특별시 강남구 논현동 142-2 정일빌딩    서울특별시 강남구 강남대로 538 (논현동)   
2    신사역성일빌딩     서울특별시 강남구 논현동 18-4 성일빌딩    서울특별시 강남구 강남대로 584 (논현동)   
3     국기원사거리   서울특별시 강남구 역삼동 648-22 동찬빌딩    서울특별시 강남구 테헤란로 125 (역삼동)   
4     대치재경빌딩      서울특별시 강남구 대치동 599 대원빌딩  서울특별시 강남구 남부순환로 2947 (대치동)   

          lat          lng phone           fax    open_dt gugun_name  \
0   37.501087   127.043069  None   02-568-3763 2019-06-13        강남구   
1   37.510178   127.022223  None  02-3442-3673 2018-11-23        강남구   
2  37.5139309  127.0206057  None   02-547-3859 2019-12-19        강남구   
3   37.499517   127.031495  None   02-568-3669 2019-07-31        강남구   
4   37.494668   127.062583  None   02-568-3705 2019-02-14        강남구   

  s_biz_code mobile_order                                          image_url  

In [ ]:
pip install requests pandas

In [ ]:
import requests
import pandas as pd
import time
from typing import List, Dict

def collect_all_starbucks_stores() -> pd.DataFrame:
    """
    스타벅스 전체 매장 정보를 수집하는 함수
    """
    base_url = "https://www.starbucks.co.kr/store/getStore.do"
    all_stores: List[Dict] = []
    page = 1
    stores_per_page = 1000

    while True:
        # 페이지네이션 파라미터 계산
        start_idx = (page - 1) * stores_per_page + 1

        # Request payload 설정
        payload = {
            'in_biz_cds': '0',
            'in_scodes': '0',
            'ins_lat': '37.5816192',
            'ins_lng': '126.681088',
            'search_text': '',
            'p_sido_cd': '01',
            'p_gugun_cd': '',
            'in_distance': '0',
            'isError': 'true',
            'searchType': 'C',
            'all_store': '0',
            'istart': start_idx,
            'iend': start_idx + stores_per_page - 1
        }

        # 요청 시도
        try:
            print(f"페이지 {page} 수집 중... (매장 {start_idx} ~ {start_idx + stores_per_page - 1})")
            response = requests.post(base_url, data=payload)
            response.raise_for_status()
            data = response.json()

            # 응답에서 매장 리스트 추출
            stores = data.get('list', [])

            # 더 이상 매장이 없으면 종료
            if not stores:
                print("더 이상 수집할 매장이 없습니다.")
                break

            all_stores.extend(stores)
            print(f"현재까지 {len(all_stores)}개 매장 수집 완료")

            # 다음 요청 전 잠시 대기
            time.sleep(1)
            page += 1

        except Exception as e:
            print(f"에러 발생: {e}")
            break

    # 수집된 데이터를 DataFrame으로 변환
    df = pd.DataFrame(all_stores)

    # 필요한 컬럼만 선택
    columns = {
        's_name': '매장명',
        'addr': '주소',
        'doro_address': '도로명주소',
        'lat': '위도',
        'lot': '경도',
        'phone': '전화번호',
        'open_dt': '오픈일자',
        'gugun_name': '구군명',
        's_biz_code': '매장코드',
        'my_siren_order_store_yn': '사이렌오더',
    }

    # 컬럼 선택 및 이름 변경
    df = df[list(columns.keys())].rename(columns=columns)

    # 데이터 정제
    df['오픈일자'] = pd.to_datetime(df['오픈일자'], format='%Y%m%d', errors='coerce')

    # 중복 제거
    df = df.drop_duplicates(subset=['매장명', '주소'], keep='first')

    return df

if __name__ == "__main__":
    print("스타벅스 매장 정보 수집을 시작합니다...")

    # 매장 정보 수집
    df = collect_all_starbucks_stores()

    # 결과 저장
    output_file = 'starbucks_stores_complete.csv'
    df.to_csv(output_file, index=False, encoding='utf-8-sig')

    print(f"\n수집 완료!")
    print(f"총 {len(df)}개 매장 정보가 {output_file}에 저장되었습니다.")
    print("\n데이터 샘플:")
    print(df.head())

스타벅스 매장 정보 수집을 시작합니다...
페이지 1 수집 중... (매장 1 ~ 1000)
더 이상 수집할 매장이 없습니다.


KeyError: "None of [Index(['s_name', 'addr', 'doro_address', 'lat', 'lot', 'phone', 'open_dt',\n       'gugun_name', 's_biz_code', 'my_siren_order_store_yn'],\n      dtype='object')] are in the [columns]"

In [ ]:
df = df[list(columns.keys())].rename(columns=columns)

NameError: name 'columns' is not defined

In [ ]:
import requests
import pandas as pd
import time
from typing import List, Dict

def collect_all_starbucks_stores() -> pd.DataFrame:
    """
    스타벅스 전체 매장 정보를 수집하는 함수
    """
    # ...

In [ ]:
df

,store_name,addr,street_address,lat,lng,phone,fax,open_dt,gugun_name,s_biz_code,mobile_order,image_url
0,역삼아레나빌딩,서울특별시 강남구 역삼동 721-13 아레나빌딩,서울특별시 강남구 언주로 425 (역삼동),37.501087,127.043069,None,02-568-3763,2019-06-13,강남구,3762,N,/upload/store/2019/06/[3762]_20190612101444_j4...
1,논현역사거리,서울특별시 강남구 논현동 142-2 정일빌딩,서울특별시 강남구 강남대로 538 (논현동),37.510178,127.022223,None,02-3442-3673,2018-11-23,강남구,3672,N,/upload/store/2018/12/[3672]_20181211120009_t9...
2,신사역성일빌딩,서울특별시 강남구 논현동 18-4 성일빌딩,서울특별시 강남구 강남대로 584 (논현동),37.5139309,127.0206057,None,02-547-3859,2019-12-19,강남구,3858,N,/upload/store/2022/01/[3858]_20220127064243_5k...
3,국기원사거리,서울특별시 강남구 역삼동 648-22 동찬빌딩,서울특별시 강남구 테헤란로 125 (역삼동),37.499517,127.031495,None,02-568-3669,2019-07-31,강남구,3669,N,/upload/store/2019/07/[3669]_20190730073527_5n...
4,대치재경빌딩,서울특별시 강남구 대치동 599 대원빌딩,서울특별시 강남구 남부순환로 2947 (대치동),37.494668,127.062583,None,02-568-3705,2019-02-14,강남구,3703,N,/upload/store/2023/12/[3703]_20231217121347_eq...
...,...,...,...,...,...,...,...,...,...,...,...,...
627,상봉역,"서울특별시 중랑구 상봉동 100-2,3,4번지",서울특별시 중랑구 망우로 307 (상봉동),37.59689,127.08647,None,02-433-8486,2012-03-23,중랑구,9686,N,/upload/store/2019/10/[9686]_20191001110003_28...
628,묵동,서울특별시 중랑구 묵동 169-12 로프트원 태릉입구역 1층,"서울특별시 중랑구 동일로 952 (묵동, 로프트원 태릉입구역) 1층",37.615368,127.076633,None,02-971-3937,2020-06-30,중랑구,3936,N,/upload/store/2020/06/[3936]_20200628090702_8k...
629,망우동,서울특별시 중랑구 망우동 488-5,서울특별시 중랑구 망우로 407 (망우동),37.5995083771241,127.097538751682,None,--,2024-11-28,중랑구,4611,N,/upload/store/2025/01/[4611]_20250102030458_sy...
630,양원역,"서울특별시 중랑구 망우동 618 B1~1층일부 B101,B102,101,102 4개호실",서울특별시 중랑구 양원역로10길 3 (망우동),37.6066536267232,127.106359790053,None,02-433-4308,2022-08-25,중랑구,4307,N,/upload/store/2022/08/[4307]_20220824060637_yg...


In [ ]:
!pip install koreanize-matplotlib

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 46.8 MB/s eta 0:00:00
